In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import polars as pl

pd.set_option('display.max_columns', None)

In [11]:
df = (
    pl.scan_csv("../data/processed/df_procesada.csv", separator=";")
    .filter((pl.col("TIPO_ACTIVIDAD") == "HOSPITALIZACIÓN"))
    .group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL"]))
    .agg(
        pl.col("DIAGNOSTICO1").count().alias("n_egresos"),
        pl.col("ESTANCIA").sum().alias("dias_estada_totales"),
        pl.col("IR_29301_PESO").mean().alias("promedio_peso_grd"),
    )
    .sort(by=["ANIO_EGRESO", "promedio_peso_grd"], descending=True)
).collect()

In [12]:
df

ANIO_EGRESO,COD_HOSPITAL,n_egresos,dias_estada_totales,promedio_peso_grd
i64,i64,u32,i64,f64
2023,112103,3594,32385,2.374888
2023,112104,3558,23945,1.875305
2023,111195,11198,102014,1.503182
2023,112102,7519,48336,1.389222
2023,112100,10721,112952,1.303186
2023,110110,3115,15961,1.301621
2023,106102,4224,48442,1.300842
2023,107100,21896,168148,1.279759
2023,110100,20198,151901,1.242069
